# Primer evaluation

This notebook evaluates the quality and specificity of primers obteined using sequence data retrieved from NCBI and processed with a bioinformatics pipeline.

## Example | Genera: *Viola*, Gene: matK 

The **matK** gene in the genus *Viola*, is used for this example.

> Input configuration (in **config/config.yml**)
>```yml
>entrez:
>  genes: ["matk"]
>  organisms: ["viola"]
>  min_len: 200
>  max_len: 2000
>primer3:
>  PRIMER_OPT_SIZE: 20
>  PRIMER_MIN_SIZE: 18
>  PRIMER_MAX_SIZE: 24
>  PRIMER_PRODUCT_SIZE_RANGE: "100-300"
>```
>   *Executed: 2025/06/17*

NCBI exploration with `Entrez` (rule: **exploration**) obtained  followed NCBIs IDs:

```raw
2838045729	2838045547	2838045453
2838045268	2820060830	2820060822
2820060818	2820060814	2629966613
2736032405	2736032403	2736032401
2736032399	2736032397	2718041013
2687757718	2502706083	2502704783
2502703935	2618954001	
```

### Determine the amplicon for de selected primers

The **primer pair** with the lowest penalty score was selected (**0.428421**), from the output file of de pipeline (generated: **data/viola_matk_primers.txt**)

| Primer | Penalt. Score | Start. Pos. | Length | Sequence             |
| ------ | ------------- | ----------- | ------ | -------------------- |
| left   | 0.253347      | 613         | 20     | TCCAAGCATTCCCTCTCCCT |
| right  | 0.175073      | 828         | 20     | ATCAGCCCGAGTCGGTTTAC |

In [1]:
# Amplicon for best primer pair
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from os.path import join, abspath, exists
from os import getcwd

genes = "matk"
genera = "viola"

PATH_CONS = abspath(join(getcwd(), "..", "data", f"{genera}_{genes}_cons.fasta"))
PATH_AMP = abspath(join(getcwd(), "..", "data", f"{genera}_{genes}_ampl.fasta"))

record = SeqIO.read(PATH_CONS, "fasta")
sequence = str(record.seq)

# Selected primers positions
left_start = 613
left_len = 20
right_start = 828
right_len = 20

left_primer = sequence[left_start:left_start + left_len]
right_primer = sequence[right_start:right_start + right_len]

amplicon = sequence[left_start:right_start + right_len]

print("\nPrimer forward (5'→3'):", left_primer)
print("Primer reverse (5'→3'):", right_primer[::-1])

print("\nPositions:")
print(f"Forward primer: {left_start}–{left_start+left_len-1}")
print(f"Reverse primer: {right_start}–{right_start+right_len-1}")

print("\nAmplicon:")
print(amplicon, "\n")

record = SeqRecord(
    Seq(amplicon),
    id="viola_amplicon",
    description="PCR product using best primer pair"
)

if not exists(PATH_AMP):
    with open(PATH_AMP, "w") as handle:
        SeqIO.write(record, handle, "fasta")



Primer forward (5'→3'): TCTTTGCATTTATTACGACT
Primer reverse (5'→3'): AAAGTATCTTTATATAAGCA

Positions:
Forward primer: 613–632
Reverse primer: 828–847

Amplicon:
TCTTTGCATTTATTACGACTCTTTCTTCATGAGTATTGGAATTTGAnnnACAGTCTTATTATTCCAAAGAAATCTATTTCCATTTTTGCAAAGGATAATCCAAGATTATTCTTGTTCTTATATAATTTTCATGTATATGAATACGAATCTATTCTCTTCTTTCTTCGTAACCAATCCTTTCATTTACAATCAACATTTTTTCGAGTCCTTTTTGAACGAATATATTTCTATGAAA 



### BLAST Search and Hit Analysis

To assess the specificity of the selected primers, a BLASTn search was conducted using the resulting amplicon against the NCBI nucleotide database (`nt`).

In [3]:
# perform a BLAST search to NCBI sequences database
from Bio.Blast import NCBIWWW
from Bio import SeqIO
import dotenv
from os import getenv, getcwd
from os.path import join, exists, abspath

dotenv.load_dotenv()
NCBIWWW.email = getenv("email")

genes = "matk"
genera = "viola"

PATH_AMP = abspath(join(getcwd(), "..", "data", f"{genera}_{genes}_ampl.fasta"))
PATH_XML_BLAST = abspath(join(getcwd(), "..", "data", f"{genera}_{genes}_blast.xml"))

amplicon = SeqIO.read(PATH_AMP, "fasta")

# store results to XML
if not exists(PATH_XML_BLAST):
    result = NCBIWWW.qblast("blastn", "nt", amplicon.seq)
    with open(PATH_XML_BLAST, "w") as handle:
        handle.write(result.read())
        result.close()

The top hits are parsed and summarized in a table for further inspection.

In [4]:
# result tabulation
from Bio.Blast import NCBIXML
import pandas as pd

with open(PATH_XML_BLAST) as result:
    blast_record = NCBIXML.read(result)

data = []
for alignment in blast_record.alignments:
    hsp = alignment.hsps[0]
    identity = (hsp.identities / hsp.align_length) * 100
    data.append({
        "Organism": alignment.hit_def,
        "Score": hsp.score,
        "E-value": hsp.expect,
        "Identity (%)": round(identity, 2),
        "Align length": hsp.align_length
    })

df_hits = pd.DataFrame(data)

df_hits.head(10)

,Organism,Score,E-value,Identity (%),Align length
0,Viola cucullata voucher AP011 maturase K (matK...,453.0,1.440620e-109,98.72,235
1,Viola elatior voucher KG23-0398 maturase K (ma...,453.0,1.440620e-109,98.72,235
2,"Viola odorata genome assembly, organelle: plas...",453.0,1.440620e-109,98.72,235
3,Viola renifolia voucher 09PROBE-05214 maturase...,453.0,1.440620e-109,98.72,235
4,Viola labradorica isolate AG2KK53 maturase K (...,453.0,1.440620e-109,98.72,235
5,"Viola odorata chloroplast, complete genome",453.0,1.440620e-109,98.72,235
6,Viola sororia isolate OSBAR 000338 maturase K ...,453.0,1.440620e-109,98.72,235
7,Viola labradorica voucher AP449 maturase K (ma...,453.0,1.440620e-109,98.72,235
8,"Viola biflora chloroplast, complete genome",453.0,1.440620e-109,98.72,235
9,"Viola verecunda chloroplast, complete genome",453.0,1.440620e-109,98.72,235


Confirm specificity using a quick filter operation

In [5]:
# hits summary
viola_hits = df_hits[df_hits["Organism"].str.contains("Viola")]
print(f"Total hits: {len(df_hits)}")
print(f"Hits matching 'Viola': {len(viola_hits)}")

viola_hits.head(10)

Total hits: 50
Hits matching 'Viola': 50


,Organism,Score,E-value,Identity (%),Align length
0,Viola cucullata voucher AP011 maturase K (matK...,453.0,1.440620e-109,98.72,235
1,Viola elatior voucher KG23-0398 maturase K (ma...,453.0,1.440620e-109,98.72,235
2,"Viola odorata genome assembly, organelle: plas...",453.0,1.440620e-109,98.72,235
3,Viola renifolia voucher 09PROBE-05214 maturase...,453.0,1.440620e-109,98.72,235
4,Viola labradorica isolate AG2KK53 maturase K (...,453.0,1.440620e-109,98.72,235
5,"Viola odorata chloroplast, complete genome",453.0,1.440620e-109,98.72,235
6,Viola sororia isolate OSBAR 000338 maturase K ...,453.0,1.440620e-109,98.72,235
7,Viola labradorica voucher AP449 maturase K (ma...,453.0,1.440620e-109,98.72,235
8,"Viola biflora chloroplast, complete genome",453.0,1.440620e-109,98.72,235
9,"Viola verecunda chloroplast, complete genome",453.0,1.440620e-109,98.72,235


### Conclusion
The BLAST results show that the amplicon generated by the best-scoring primers aligns almost exclusively with sequences from the genus *Viola*, suggesting a high degree of specificity.

This indicates that the **primer3-based pipeline** performs well for this target region, at least in silico. Further experimental validation (e.g., PCR tests with DNA from *Viola* and non-*Viola* species) is needed to confirm these results in vitro.

This step completes the initial validation phase and supports the pipeline's ability to design genus-specific primers.